In [42]:
from scipy.optimize import fsolve
import numpy as np

In [43]:
def reynoldsNumber(density, velocity, diameter, viscosity) -> float:
    """
    Return the reynold's number
    """
    den = density
    vel = velocity
    dia = diameter
    vis = viscosity
    reynolds_number = (den*vel*dia)/vis
    print("Reynold's number ", reynolds_number)
    return reynolds_number

In [44]:

# This is the rearranged expression from above
def frictionFactor(reynolds_number, rel_roughness, diameter):
    """
    Return the friction factor
    """
    rey = reynolds_number
    rel = rel_roughness
    dia = diameter
    def calculateFrictionFactor(x):
        return (1/x**0.5) + 2*np.log10(((rel/dia))/3.7 + (2.51/(rey*x**0.5)))

    # Solve the equation
    # plug in the initial guess in second paramter
    solution = fsolve(calculateFrictionFactor, 0.02)
    print("Friction factor ", solution[0])
    return solution[0]

In [45]:
def getV2(z_2, z_1, friction_factor, length, diameter, p_2, v2_guess):
    """
    Calculate V_2
    """
    fri = friction_factor
    dia = diameter
    h = z_1 # height
    g = 9.81
    alpha = 1
    k = 0.5
    p_atm = 101.325
    roh = 1000
    a_2 = (np.pi * (diameter/2)**2) # area of pipe
    a_1 = 0.32 * 0.26 # area of block
    # let x = v_2
    def calculateV2(x):
        equation = (p_atm/roh*g) + ((alpha*((a_2/a_1) * x)**2)/(2*g)) + z_1 - ((p_2 + (roh * g * h))/(roh * g)) - ((alpha * x ** 2)/(2*g)) - z_2 
        - (fri * (length / dia) * ((x**2)/2*g)) - ((k*x**2)/(2*g))
        return equation
    
    solution = fsolve(calculateV2, v2_guess)
    print("Calculated v_2 ", solution[0])
    return solution[0]

In [48]:
# set constants
diameter = 0.00794
tube_diameter = 0.00794
density_water = 1000
viscosity = 1.0005
rel_roughness = 7*10**-6
z_1 = 0
p_atm = 101.325
p_2 = p_atm

# set variables
v2_guess = 10
tube_length = 0.60
# z_2 is start height of water
z_2 = 0.08 + 0.02 + (1/150 * tube_length)


# calculate reynold's number
reynolds_number = reynoldsNumber(density_water, v2_guess, diameter, viscosity)

# calculate friction factor
friction_factor = frictionFactor(reynolds_number, rel_roughness, diameter)

# calculate the new v_2
calculated_v2 = getV2(z_2, z_1, friction_factor, tube_length, tube_diameter, p_2, v2_guess)

# initial tolerance
tol = 10

# fixed point iteration
while tol>0.000001:
    print("\n")
    print("v2_guess ", v2_guess, "calculated_v2 ", calculated_v2)
    v2_guess = calculated_v2
    reynolds_number = reynoldsNumber(density_water, v2_guess, diameter, viscosity)
    friction_factor = frictionFactor(reynolds_number, rel_roughness, diameter)
    calculated_v2 = getV2(z_2, z_1, friction_factor, tube_length, tube_diameter, p_2, v2_guess)
    tol = abs(v2_guess - calculated_v2)

print("final v2 ", calculated_v2, "guess ", v2_guess, "tol ", tol)

Reynold's number  79.36031984007995
Friction factor  0.1922294318332183
Calculated v_2  4.154409919315894


v2_guess  10 calculated_v2  4.154409919315894
Reynold's number  32.96952999437101
Friction factor  0.32671785760881616
Calculated v_2  4.154409919315894
final v2  4.154409919315894 guess  4.154409919315894 tol  0.0
